# ==========================

# HRV analysis with patients' main dx (Diagnosed disease class)

* Dataset provided by SMC Professor J.A.
* Research Goal: Estimate or predict depressed/anxious status based on HRV dataset

In [ ]:
import random
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# import keras_one_cycle_clr as ktool

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# import torchvision

from PIL import Image

In [ ]:
from scipy import stats
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense , Activation, Dropout, BatchNormalization
from keras.optimizers import Adam ,RMSprop
from keras import  backend as K
from keras.optimizers import SGD
# from tensorflow.keras import utils as np_utils
# from tensorflow.keras.metrics import binary_focal_crossentropy
from sklearn import decomposition, metrics
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
# from torch.utils.data import TensorDataset, DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler, RobustScaler, Normalizer

# from tqdm import tqdm
# from torch.autograd import Variable
# from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import cohen_kappa_score,f1_score, confusion_matrix
from sklearn.model_selection import KFold, train_test_split
from keras.callbacks import Callback
# from pytorch_tabnet.tab_model import TabNetClassifier

In [ ]:
# ## Set this if you want to check all information from dataframe without ... auto.
# np.set_printoptions(threshold=np.inf, linewidth=np.inf) #inf = infinity 
# pd.set_option('display.max_rows', 100)
# pd.set_option('display.max_columns', 100)
# pd.set_option('display.width', 100)

# ==========================

# Data Handling

* 주요 outlier:
> 1. SRD 0.8 미만 혹은 1.0 초과
> 2. 이상심박동수(abnormal_hr) 5회 초과

In [ ]:
hrv_ori = pd.read_csv('E:/RESEARCH/Datasets/HRV/JA/HRV_prep.csv')

In [ ]:
hrv_ori.shape

* 기기상의 오류로 인한 결측치를 제거해주자.

In [ ]:
hrv_ori = hrv_ori[(hrv_ori.srd >=0.8) & (hrv_ori.srd <= 1.0)] ## srd outlier removed - there were 1424 outlier data
hrv_ori = hrv_ori[(hrv_ori.abnormal_hr) <= 5] ## abnormal heart rate outlier removed - there were 175 outlier data

In [ ]:
hrv_ori.shape ## total 1491 data removed (outlier due to the hardware error)

In [ ]:
hrv_ori["main_dx"].value_counts() ## checking the numbers for each main_dx classification
# hrv_ori["second_dx"].value_counts() ## checking the numbers for each second_dx classification
# hrv_ori["third_dx"].value_counts() ## checking the numbers for each third_dx classification

In [ ]:
# print(hrv_ori.dtypes)

* 본 연구에서 확인하고자 하는 주요 질병들은 MDD(s,r), PDD, BP(I,II), 
> 따라서 각 질병 별로 남기고 나머지는 제거하는 작업 필요.

In [ ]:
hrv_ori['bipolar'] = 0 ##  first, make the empty variable called "bipolar"

In [ ]:
hrv_ori.loc[(hrv_ori['main_dx']=="BP_I"), 'bipolar' ] = 'one'
hrv_ori.loc[(hrv_ori['main_dx']=="BP_II"), 'bipolar'] = 'two'

In [ ]:
hrv_ori.head()

In [ ]:
bipolar_empty = hrv_ori[hrv_ori['bipolar']==0].index
hrv_ori_bp = hrv_ori.drop(bipolar_empty)

In [ ]:
hrv_ori_bp.head()

In [ ]:
print("The shape of the processed bipolar disorder patients' hrv dataset is", hrv_ori_bp.shape)

In [ ]:
hrv_ori_bp["bipolar"].value_counts()

In [ ]:
hrv_ori = hrv_ori_bp

In [ ]:
# hrv_ori.describe()

In [ ]:
# hrv_ori["HAMD"].describe()
# hrv_ori["HAMA"].describe()
# hrv_ori["BDI-II"].describe()
# hrv_ori["BAI"].describe()
# hrv_ori["gender"].value_counts()

## Gender, Age Separation

### 1) Gender

* select the "Female" dataset only

In [ ]:
# hrv_ori = hrv_ori[hrv_ori.gender == "F"]

* select the "Male" dataset only

In [ ]:
# hrv_ori = hrv_ori[hrv_ori.gender == "M"]

In [ ]:
hrv_ori.head()

### 2)  Age

In [ ]:
# hrv_ori = hrv_ori[(hrv_ori.age >=20) & (hrv_ori.age < 30)]

-------

## Data Separation

### Dataset variables

> "Demographic info" : Gender, Age \
> "Questionnaires" : HAMD, HAMA, BDI-II, BAI, MDQ, HCL-32 \
> "HRV's 17 Features" : SDNN, PSI, VLF, LF, HF, TP, LFNORM, HFNORM, LF/HF Ratio, RMSSD, APEN, SRD, TSRD, TP_ln, LF_ln, HF_ln \
> "Current Status labels":
>> HAMD_ : Hamilton Depression Rating Scale \
>> HAMA_ : Hamilton Anxiety Rating Scale \
>> BDI_ : Beck Depression Index \
>> BAI_ : Beck Anxiety Index \

In [ ]:
hrv_feature_list = ['sdnn','psi','tp','vlf','lf','hf','lfnorm','hfnorm','lf_hf','rmssd','apen','srd','tsrd','tp_ln', 'vlf_ln','lf_ln','hf_ln']
hrv_feature_list_core = ['sdnn', 'tp','vlf','lfnorm','hfnorm','rmssd','apen','srd','tsrd']

In [ ]:
hrv_bp = hrv_ori.loc[:, ['sdnn','psi','tp','vlf','lf','hf','lfnorm','hfnorm','lf_hf','rmssd','apen','srd','tsrd','tp_ln', 'vlf_ln','lf_ln','hf_ln','bipolar']] ## all HRV features
hrv_bp_core = hrv_ori.loc[:, ['sdnn', 'tp','vlf','lfnorm','hfnorm','rmssd','apen','srd','rmssd','bipolar']] ## core HRV features only

In [ ]:
hrv_bp_multi = hrv_ori.loc[:, ['sdnn','psi','tp','vlf','lf','hf','lfnorm','hfnorm','lf_hf','rmssd','apen','srd','tsrd','tp_ln', 'vlf_ln','lf_ln','hf_ln',
                                 'auto_activity','auto_balance','stress_resist','stress_index','tired','avg_hr','heart_stable','abnormal_hr','bipolar']]

* Delete the patients with null value.

In [ ]:
hrv_bp = hrv_bp.dropna(subset = ['bipolar'], axis=0)
hrv_bp_core = hrv_bp_core.dropna(subset = ['bipolar'], axis=0)

## Multimodality approach

In [ ]:
hrv_bp_multi = hrv_ori.loc[:, ['sdnn','psi','tp','vlf','lf','hf','lfnorm','hfnorm','lf_hf','rmssd','apen','srd','tsrd','tp_ln', 'vlf_ln','lf_ln','hf_ln',
                                 'auto_activity','auto_balance','stress_resist','stress_index','tired','avg_hr','heart_stable','abnormal_hr','bipolar']]

In [ ]:
hrv_bp_multi = hrv_bp_multi.dropna(subset = ['bipolar'], axis=0)

In [ ]:
# hrv_bp_multi

---------

# ==========================

## Data Selection

* Select the dataset for further analysis

In [ ]:
data = hrv_bp

In [ ]:
data.shape

In [ ]:
data

In [ ]:
data["bipolar"].value_counts()

----------

## Data preprocessing

### Removing HRV outliers

In [ ]:
# feature_list = ['sdnn', 'psi', 'tp', 'vlf', 'lf', 'hf', 'lfnorm', 'hfnorm', 'lf_hf', 'rmssd', 'apen', 'srd', 'tsrd', 'tp_ln', 'vlf_ln', 'lf_ln', 'hf_ln']
# data_outlier = data.copy()

In [ ]:
# var = 'HAMD_'
# var = 'HAMA_'
# var = 'BDI_'
# var = 'BAI_'
# var = 'depression'
# var = 'anxious'
var = 'bipolar'

In [ ]:
data[var].value_counts()

* severity classification (4classes)

In [ ]:
normal = data[data[var]=='normal']
mild = data[data[var]=='mild']
moderate = data[data[var]=='moderate']
severe = data[data[var]=='severe']

* Subtype classification (binary)

In [ ]:
type_one = data[data[var]=='one']
type_two = data[data[var]=='two']

In [ ]:
## check sample dataset
# type_one

#### Removing outliers with q1, q2 quantile values

In [ ]:
# ## Removing outlier of each HRV feature variables.
# for feature in feature_list:
#     q1 = data[feature].quantile(0.25)
#     q3 = data[feature].quantile(0.75)
#     iqr = q3 - q1
#     condition = data[feature]>q3+(1.5*iqr)
#     ind = data[condition].index
#     data.drop(ind, inplace=True)

In [ ]:
data.shape

### Outliers check with boxplot

In [ ]:
normal = data[data[var]=='normal']
mild = data[data[var]=='mild']
moderate = data[data[var]=='moderate']
severe = data[data[var]=='severe']

In [ ]:
data_box = data.loc[:, ['sdnn','psi','tp','vlf','lf','hf','lfnorm','hfnorm','lf_hf','rmssd','apen',
                                  'srd','tsrd', 'tp_ln', 'vlf_ln','lf_ln','hf_ln']]
normal_box = normal.loc[:, ['sdnn','psi','tp','vlf','lf','hf','lfnorm','hfnorm','lf_hf','rmssd','apen',
                                  'srd','tsrd', 'tp_ln', 'vlf_ln','lf_ln','hf_ln']]
mild_box = mild.loc[:, ['sdnn','psi','tp','vlf','lf','hf','lfnorm','hfnorm','lf_hf','rmssd','apen',
                                  'srd','tsrd', 'tp_ln', 'vlf_ln','lf_ln','hf_ln']]
moderate_box = moderate.loc[:, ['sdnn','psi','tp','vlf','lf','hf','lfnorm','hfnorm','lf_hf','rmssd','apen',
                                  'srd','tsrd', 'tp_ln', 'vlf_ln','lf_ln','hf_ln']]
severe_box = severe.loc[:, ['sdnn','psi','tp','vlf','lf','hf','lfnorm','hfnorm','lf_hf','rmssd','apen',
                                  'srd','tsrd', 'tp_ln', 'vlf_ln','lf_ln','hf_ln']]

In [ ]:
scaler = MinMaxScaler()
# scaler = RobustScaler()
# scaler = Normalizer()

In [ ]:
data_box[:] = scaler.fit_transform(data_box[:])
normal_box[:] = scaler.fit_transform(normal_box[:])
mild_box[:] = scaler.fit_transform(mild_box[:])
moderate_box[:] = scaler.fit_transform(moderate_box[:])
severe_box[:] = scaler.fit_transform(severe_box[:])

In [ ]:
# fig, ax = plt.subplots()
plt.figure(figsize = (10, 5))
sns.set_style("whitegrid")
plt.xlabel('HRV feature variables',fontsize=10)
plt.ylabel('Feature value',fontsize=10)
plt.boxplot(data_box)
plt.show()

In [ ]:
# fig, ax = plt.subplots()
plt.figure(figsize = (10, 5))
plt.xlabel('HRV feature variables',fontsize=10)
plt.ylabel('Feature value',fontsize=10)
plt.boxplot(normal_box)
plt.show()

In [ ]:
# fig, ax = plt.subplots()
plt.figure(figsize = (10, 5))
plt.xlabel('HRV feature variables',fontsize=10)
plt.ylabel('Feature value',fontsize=10)
plt.boxplot(mild_box)
plt.show()

In [ ]:
# fig, ax = plt.subplots()
plt.figure(figsize = (10, 5))
plt.xlabel('HRV feature variables',fontsize=10)
plt.ylabel('Feature value',fontsize=10)
plt.boxplot(moderate_box)
plt.show()

In [ ]:
# fig, ax = plt.subplots()
plt.figure(figsize = (10, 5))
plt.xlabel('HRV feature variables',fontsize=10)
plt.ylabel('Feature value',fontsize=10)
plt.boxplot(severe_box)
plt.show()

# ==========================

# Statistical Check and feature selection

In [ ]:
var = 'bipolar'

In [ ]:
data_y = data.loc[:,[var]]

In [ ]:
data_x = data.loc[:, ['sdnn','psi','tp','vlf','lf','hf','lfnorm','hfnorm','lf_hf','rmssd','apen','srd','tsrd','tp_ln', 'vlf_ln','lf_ln','hf_ln']]
# data_x = data.loc[:, ['sdnn','psi','tp','vlf','lf','hf','lfnorm','hfnorm','lf_hf','rmssd','apen','srd','tsrd','tp_ln', 'vlf_ln','lf_ln','hf_ln',
#                                  'auto_activity','auto_balance','stress_resist','stress_index','tired','avg_hr','heart_stable','abnormal_hr']] ##for multimodality analysis
# data_x = data.loc[:, ['sdnn', 'tp', 'lf_hf', 'rmssd',  'srd', 'tsrd', 'tp_ln', 'vlf_ln', 'lf_ln', 'hf_ln']]
# data_x = data.loc[:,['sdnn', 'tp','vlf','lfnorm','hfnorm','rmssd','apen','srd','rmssd']]

## Scaler

In [ ]:
# scaler = MinMaxScaler()
# scaler = RobustScaler()
# scaler = Normalizer()

In [ ]:
# # scaler to normalize/standardize whole dataset
# data_x[:] = scaler.fit_transform(data_x[:])

In [ ]:
data_var = ['sdnn', 'tp', 'vlf', 'lf', 'hf', 'lfnorm', 'hfnorm', 'lf_hf', 'rmssd', 'apen', 'srd', 'tsrd', 'tp_ln', 'vlf_ln', 'lf_ln', 'hf_ln']
# data_var = ['sdnn','psi','tp','vlf','lf','hf','lfnorm','hfnorm','lf_hf','rmssd','apen','srd','tsrd','tp_ln', 'vlf_ln','lf_ln','hf_ln',
#             'auto_activity','auto_balance','stress_resist','stress_index','tired','avg_hr','heart_stable','abnormal_hr']
# data_var_ = ['sdnn', 'tp', 'lf_hf', 'rmssd',  'srd', 'tsrd', 'tp_ln', 'vlf_ln', 'lf_ln', 'hf_ln']

## T-TEST, ANOVA

In [ ]:
bp_one = data[data['bipolar']=='one']
bp_two = data[data['bipolar']=='two']

In [ ]:
## Comparing 4 groups (normal, mild, moderate, severe) with ANOVA test
stat_result =[]
for var in data_var:
    a = bp_one[var].values
    b = bp_two[var].values
    f_val , p_val = stats.f_oneway(a, b)
    stat_result.append([f_val, p_val])

In [ ]:
stat_result = pd.DataFrame(stat_result, columns = ['F-value', 'p-value'])
stat_result = stat_result.assign(HRV_feature = data_var)

In [ ]:
stat_result

* Significant variables: SDNN, LF, HF, RMSSD, SRD, TSRD, TP, VLF

In [ ]:
# data_x = data.loc[:, ['sdnn','psi','tp','lf','hf','lfnorm','hfnorm','lf_hf','rmssd','srd','tsrd','tp_ln', 'vlf_ln','lf_ln','hf_ln']]
data_x = data.loc[:, ['sdnn','tp_ln', 'vlf_ln','lf_ln','hf_ln']]
# data_x = data.loc[:, ['lf','hf','lf_hf','rmssd','tsrd','tp_ln', 'vlf_ln','lf_ln','hf_ln','auto_activity','stress_resist','stress_index','tired','avg_hr','heart_stable']]

# ==========================

# Machine Learning approaches

In [ ]:
class Args:
    # arugments
    epochs=500
    bs=64
    lr=0.001
    momentum=0.5
    num_classes=2
    verbose='store_true'
    seed=710674

args = Args()    

# np.random.seed(args.seed)
# random.seed(args.seed)
# torch.manual_seed(args.seed)

In [ ]:
# print(data_x.dtypes)

---------

### Determine the number of classes(for prediction)

In [ ]:
## Using 4 classes (normal vs mild vs moderate vs severe)
label = data_y
label = label.replace({'one': 0})
label = label.replace({'two': 1})

In [ ]:
# ## Using 3 classes (normal vs mild vs moderate/severe)
# label = data_y
# label = label.replace({'normal': 0})
# label = label.replace({'mild': 1})
# label = label.replace({'moderate': 2})
# label = label.replace({'severe': 2})

In [ ]:
# ## Using 2 classes (normal vs depression)
# label = data_y
# label = label.replace({'normal': 0})
# label = label.replace({'mild': 1})
# label = label.replace({'moderate': 1})
# label = label.replace({'severe': 1})

-----------

In [ ]:
label.value_counts()

* Convert the y values into categorical variables

In [ ]:
y = to_categorical((label), 2)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_x, y, test_size = 0.2, random_state = 710674)

In [ ]:
print("The size of training dataset is:", x_train.shape[0])
print("The size of test dataset is:", x_test.shape[0])

------

* For K-FOLD practice

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_x, y, test_size = 0.2, random_state = 710674)

In [ ]:
inputs = np.concatenate((x_train, x_test), axis = 0)
targets = np.concatenate((y_train, y_test), axis = 0)

In [ ]:
inputs.shape

In [ ]:
targets.shape

----------------

### Model generation, selection

In [ ]:
## Generate the model
model = Sequential()
model.add(Dense(256, input_dim = x_train.shape[1], activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.2)) #drop out
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(args.num_classes, activation = 'sigmoid'))

In [ ]:
# Generate the model-3
model = Sequential()
model.add(Dense(128, input_dim = x_train.shape[1], activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(1024, activation = 'relu'))
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.5)) #drop out
model.add(Dense(1024, activation = 'relu'))
model.add(Dense(1024, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(args.num_classes, activation = 'softmax'))

In [ ]:
# model.summary()

* Model 학습과정

In [ ]:
# opt = keras.optimizers.SGD(learning_rate = args.lr, decay = 1e-5, momentum = args.momentum)
# model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
# model.fit(inputs[train], targets[train], batch_size = args.bs, epochs = args.epochs, verbose = 0, class_weight = class_weight)

In [ ]:
# scores = model.evaluate(inputs[test], targets[test])
# print(f'Score for fold {fold_num}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

* 5-FOLD CROSS VALIDATION approach

In [ ]:
fold_num = 1
split_num = 5
opt = keras.optimizers.SGD(learning_rate = args.lr, decay = 1e-6, momentum = args.momentum)
# kfold = KFold(n_splits = split_num, shuffle = True)
kfold = KFold(n_splits = split_num, shuffle = False)

In [ ]:
label.value_counts()

In [ ]:
class_weight = {1:1, 0:6}

In [ ]:
acc_per_fold = []
loss_per_fold = []

In [ ]:
for train, test in kfold.split(inputs, targets):
    model = Sequential()
    model.add(Dense(128, input_dim = x_train.shape[1], activation = 'relu'))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(1024, activation = 'relu'))
    # model.add(Dropout(0.5)) #drop out
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(1024, activation = 'relu')) 
    model.add(Dense(512, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dense(128, activation = 'relu')) # added
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(args.num_classes, activation = 'softmax'))
    
    ## model compile
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    
    print('----------------------------------------')
    print(f'Training or fold {fold_num} ... ')
    
    ## fit data to model
    history = model.fit(inputs[train], targets[train], batch_size = args.bs, epochs = args.epochs, verbose = 0, class_weight = class_weight)
    
    ## generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test])
    print(f'Score for fold {fold_num}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    ## increasing fold number
    fold_num = fold_num + 1
    
    
    
## Summarizing the results
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'>> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'>>> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'>>> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
# for train, test in kfold.split(inputs, targets):
#     model = Sequential()
#     model.add(Dense(128, input_dim = x_train.shape[1], activation = 'relu'))
#     model.add(Dense(256, activation = 'relu'))
#     model.add(Dense(512, activation = 'relu'))
# #     model.add(BatchNormalization())
# #     model.add(Activation('relu'))
#     model.add(Dense(512, activation = 'relu'))
#     model.add(Dense(256, activation = 'relu'))
#     model.add(Dense(256, activation = 'relu'))
#     model.add(Dense(128, activation = 'relu')) # added
#     model.add(Dense(args.num_classes, activation = 'softmax'))
    
#     ## model compile
#     model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    
#     print('----------------------------------------')
#     print(f'Training or fold {fold_num} ... ')
    
#     ## fit data to model
#     history = model.fit(inputs[train], targets[train], batch_size = args.bs, epochs = args.epochs, verbose = 0, class_weight = class_weight)
    
#     ## generate generalization metrics
#     scores = model.evaluate(inputs[test], targets[test])
#     print(f'Score for fold {fold_num}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
#     print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
#     acc_per_fold.append(scores[1] * 100)
#     loss_per_fold.append(scores[0])
    
#     ## increasing fold number
#     fold_num = fold_num + 1

    
    
    
# ## Summarizing the results
# print('------------------------------------------------------------------------')
# print('Score per fold')
# for i in range(0, len(acc_per_fold)):
#     print('------------------------------------------------------------------------')
#     print(f'>> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
# print('------------------------------------------------------------------------')
# print('Average scores for all folds:')
# print(f'>>> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
# print(f'>>> Loss: {np.mean(loss_per_fold)}')
# print('------------------------------------------------------------------------')

In [ ]:
# print("Average classification accuracy is", (sum(acc_per_fold)/split_num))
# # print("Average classification loss is", (sum(loss_per_fold)/5))

In [ ]:
y_predict = model.predict(x_test)
y_predict = np.argmax(y_predict, axis = 1)
y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(5, 3))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

* sklearn.metrics 내의 함수를 통해 evaluation score 들을 파악해보자.
> 1. accuracy = (TP + TN) / (TP + TN + FP + FN) 
> 2. precision = TP / (TP + FP)
> 3. recall = TP / (TP + FN) 
> 4. f1 = (2 * precision * recall) / (precision + recall)

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("=============================================")

In [ ]:
# model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

* Setting class weight to reduce class imbalance issue

In [ ]:
label.value_counts()

In [ ]:
# class_weight = {1:1, 0:5}
class_weight = {1:1, 2: 1.1 , 0:5}

In [ ]:
model.fit(x_train, y_train, epochs = args.epochs, batch_size = args.bs, verbose=0, class_weight = class_weight)

In [ ]:
scores = model.evaluate(x_test, y_test)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-------------

----------

# ==========================

# Data Augmentation 

## 1. Using accessible Public Dataset

### 1) Date & feature selection

* 일단은 위에서 사용한 core dataset을 선정하자
1. RDATA = Real dataset (실제 분석에 사용하고자 하는 dataset)
2. PDATA = Public dataset (data augmentation을 위한 public dataset)

In [ ]:
RDATA_ = hrv_ori.loc[:, ['rmssd', 'avg_hr', 'vlf', 'lf', 'hf', 'tp', 'lf_hf', 'HAMD_']]
RDATA = RDATA_.drop(['HAMD_'], axis=1)

In [ ]:
PDATA_ = pd.read_csv('E:/RESEARCH/Datasets/HRV/HRV_Public/SWELL_hrv/data/final/train_for_augmentation.csv', sep=',')

In [ ]:
print("Original data shape is", RDATA_.shape)
print("Data for augmentation shape is", PDATA_.shape)

* SWELL dataset에는 세가지 status에 해당하는 HRV 값들이 있다. (no stress, interruption, time pressure)

In [ ]:
PDATA_base = PDATA_.loc[PDATA_.condition == 'no stress']
PDATA_stress = PDATA_.loc[(PDATA_.condition =='interruption') | (PDATA_.condition =='time pressure')]

* augmentation 에 사용하고자 하는 데이터 양이 너무 많아서 약간 줄여주자

In [ ]:
PDATA_ = PDATA_stress.sample(n=3000)

In [ ]:
# sns.pairplot(PDATA)

In [ ]:
# 일단 PDATA는 unlabeled dataset이라고 생각해주고 현재 라벨 버린 뒤, -1로 라벨값을 만들어주고,
PDATA = PDATA_.drop(['condition'], axis=1)
PDATA['y'] = -1

In [ ]:
PDATA.info()

* Real data에 어떤 변수들이 있는지 확인.


In [ ]:
RDATA.info()

* Real Data의 컬럼명을 PDATA랑 맞춰주자.

In [ ]:
RDATA.columns = ['RMSSD', 'HR', 'VLF', 'LF', 'HF', 'TP', 'LF_HF']

* Real data에서 target으로 삼을 label을 따로 추출해서 범주화

In [ ]:
target_ = RDATA_.loc[:, 'HAMD_']

In [ ]:
target_.shape

In [ ]:
## Using 4 classes (normal vs mild vs moderate vs severe)
target = target_
target = target.replace({'normal': 0})
target = target.replace({'mild': 1})
target = target.replace({'moderate': 2})
target = target.replace({'severe': 3})

In [ ]:
target.value_counts()

* Data augmentation을 위해 변수값들을 표준화 시켜주는 과정

In [ ]:
scaler = MinMaxScaler() #set the scaler

RDATA[:] = scaler.fit_transform(RDATA[:])
RDATA = RDATA.round(decimals=2)
PDATA[:] = scaler.fit_transform(PDATA[:])
PDATA = PDATA.round(decimals=2)

* 표준화하니까 데이터가 차원축소했을 때 설명력이 떨어져서 그냥 주석처리

* Real data는 차원축소를 진행해도 설명력을 충분히 가지는지 확인.

In [ ]:
Rpca_3 = decomposition.PCA(n_components=3)
Rpca_3_result = Rpca_3.fit_transform(RDATA)
Rtotal_var3 = Rpca_3.explained_variance_ratio_.sum()*100

# PCA를 통해 축소된 3차원이 어느정도의 대표성을 가지는지 확인하기 위함.
print('Explained variation per principal component: {}'.format(pca_3.explained_variance_ratio_))
print('Cumulative variance explained by 3 principal components: {:.2%}'.format(np.sum(Rpca_3.explained_variance_ratio_)))

In [ ]:
RDATA_pca = pd.DataFrame(Rpca_3.transform(RDATA), columns = ['PCA%i' % i for i in range(3)], index = RDATA.index)

In [ ]:
# Plot initialisation
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='3d')
plt.title('PCA 3 result from Real HRV dataset', fontsize=12, fontweight='bold')
ax.scatter(RDATA_pca['PCA0'], RDATA_pca['PCA1'], RDATA_pca['PCA2'], cmap="Set2_r", s=60)
# plt.savefig('pca_result.png')

* 마찬가지로 augmentation을 위한 public data도 차원축소를 해도 설명력을 가지는지 확인.

In [ ]:
PDATA_ = PDATA.drop(['y'], axis = 1)

In [ ]:
Ppca_3 = decomposition.PCA(n_components=3)
Ppca_3_result = Ppca_3.fit_transform(PDATA_)
total_var3 = Ppca_3.explained_variance_ratio_.sum()*100

# PCA를 통해 축소된 3차원이 어느정도의 대표성을 가지는지 확인하기 위함.
print('Explained variation per principal component: {}'.format(Ppca_3.explained_variance_ratio_))
print('Cumulative variance explained by 3 principal components: {:.2%}'.format(np.sum(Ppca_3.explained_variance_ratio_)))

In [ ]:
PDATA_pca = pd.DataFrame(Ppca_3.transform(PDATA_), columns = ['PCA%i' % i for i in range(3)], index = PDATA.index)

In [ ]:
# Plot initialisation
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='3d')
plt.title('PCA 3 result from Public HRV dataset', fontsize=12, fontweight='bold')
ax.scatter(PDATA_pca['PCA0'], PDATA_pca['PCA1'], PDATA_pca['PCA2'], cmap="Set2_r", s=60)
# plt.savefig('pca_result.png')

* Augmentation에 쓸 데이터를 나눠놓자. (real data 먼저)

In [ ]:
# Labeled datapoints and following labels.
x1, x3 = np.split(RDATA, [int(0.2*len(RDATA))])
y1, y3 = np.split(target, [int(0.2*len(target))])

In [ ]:
print(x1.shape)
print(x3.shape)
print(y1.shape)
print(y3.shape)

* 이번에는 public data체크하자.

In [ ]:
# Unlabeled datapoints and following labels.
x2 = PDATA.loc[:,['RMSSD', 'HR', 'VLF', 'LF', 'HF', 'TP', 'LF_HF']]
y2 = PDATA['y']

In [ ]:
print(x2.shape)
print(y2.shape)

* Real data(training 부분만)랑 Public data 합치자.

In [ ]:
# Concatenate
x12 = np.concatenate((x1, x2))
y12 = np.concatenate((y1, y2))

* Logistic Regression 통해서 라벨 데이터(Real dataset)을 체크해보자.

In [ ]:
index = ['Analysis Method', 'ROC AUC']
results = pd.DataFrame(columns = index)

In [ ]:
np.isnan(x1.any())

In [ ]:
np.any(np.isnan(x1))

In [ ]:
np.all(np.isfinite(x1))

In [ ]:
logreg = LogisticRegression(random_state = 710674, class_weight = 'balanced')
logreg.fit(x1, y1)
results = results.append(
    pd.Series(['Logistic Regression', roc_auc_score(y3, logreg.predict_proba(x3), multi_class='ovr')],
              index=index), ignore_index=True)

In [ ]:
y1

## 2. Using its own Dataset